In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime

## PRZEMYŚLENIA
czy w outlecie powinno być id_inventory z rentala czy id_gry



----

pełną dokumentację funkcji zrobi się później

---

In [2]:
t = 10000 ##żeby wykluczyć mało znane imiona i nazwiska

##imiona męskie i kobiece
im = pd.read_csv("IM.csv")
im = im[im["LICZBA WYSTĄPIEŃ"] > t] 
ik = pd.read_csv("IK.csv")
ik = ik[ik["LICZBA_WYSTĄPIEŃ"] > t]
ik = ik["IMIĘ_PIERWSZE"]
im = im["IMIĘ PIERWSZE"]

##nazwiska męskie i kobiece
nm = pd.read_csv("NM.csv")
nm = nm[nm["Liczba"] > t]
nk = pd.read_csv("NK.csv")
nk = nk[nk["Liczba"] > t]
nk = nk["Nazwisko aktualne"]
nm = nm["Nazwisko aktualne"]

ulice = pd.read_csv("ULIC.csv", sep=";")
ulice_jaworze = ulice.query('WOJ == 24 & POW == 2 & GMI == 6 & RODZ_GMI == 2') ##bo kod terytorialny jaworza to 24.2.6.2
ulice_jaworze = np.array(ulice_jaworze["NAZWA_1"]) ##i bierzemy ulice

##gry
gry = pd.read_csv("GRY.csv")
gry

,ID,NAZWA,RODZAJ,CENA,CZAS_GRY,MIN_GRACZY,MAX_GRACZY,MIN_WIEK,TURNIEJOWE
0,1,Brass: Birmingham,"ekonomiczna, strategiczna",250,150,2,4,14,0
1,2,Star Wars: Rebelia,"strategiczna, science-fiction",390,210,2,4,14,0
2,3,Wojna o Pierścień: Druga edycja,"przygodowa, fantasy",330,150,2,4,13,0
3,4,Zaginiona Wyspa Arnak,"przygodowa, eksploracyjna",150,90,1,4,12,0
4,5,Zamki Burgundii: BIG BOX,strategiczna,140,60,2,4,12,0
...,...,...,...,...,...,...,...,...,...
95,96,Eurobusiness,"ekonomiczna, rywalizacyjna",30,90,2,5,14,0
96,97,Boop,"logiczna, rywalizacyjna",160,20,2,2,10,0
97,98,Mordercze krewetki,"karciane, towarzyskie",80,10,2,6,7,0
98,99,The binding of Isaac: Four souls,"przygodowa, karciana",190,45,2,4,13,0


In [58]:
gry = gry.rename(columns={"ID":"id_gry", "NAZWA":"tytuł", "RODZAJ":"rodzaj", "CENA":"cena", "CZAS_GRY":"czas_gry", 
                         "MIN_GRACZY":"min_graczy", "MAX_GRACZY":"max_graczy", "MIN_WIEK":"min_wiek", "TURNIEJOWE":"turniejowe"})

In [3]:
def ludzie(mi, ki, mn, kn, ul, pk= 0.516, N=7349 ):
    """
    Funkcja do generowania ludności z Jaworza
    pk = 0.516 - procent kobiet w społeczeństwie
    N = 7349 - (w miarę) aktualny stan ludności w tej urokliwej wsi
    
    """
    
    imie = []
    nazwisko = []
    wiek = []
    adres = []
    tel = []
    
    for n in range(N):
        
        p = random.random()
        
        if p < pk: ##losujemy kobietę
            imie.append(random.choice(ki).title()) ##title z AAAA robi Aaaa (a taką formę imion mamy)
            nazwisko.append(random.choice(kn).title())
        
        else: #losujemy mężczyznę
            imie.append(random.choice(mi).title())
            nazwisko.append(random.choice(mn).title())
        
        p = random.random()
        
        if p < 0.575: ##dane statystyczne - 57.5% osób jest w wieku produkcyjnym
            wiek.append(random.randint(18,60))
        elif p < 0.77:
            wiek.append(random.randint(8, 18)) ##19.4% w przedprodukcyjnym
        else:
            wiek.append(random.randint(60,99)) ##reszta starsza
            
            
        p = random.random()   
        if p< 0.90:
            adres.append(random.choice(ul) + ' ' + str(random.randint(1,100)))
            
            p = random.random()
            if p< 0.90:
                tel.append(random.randint(500000000, 999999999))
            else:
                tel.append("NULL")
                
        else:
            adres.append("NULL")
            tel.append(random.randint(500000000, 999999999))
        
        
            
    
    return pd.DataFrame({"id_mieszkańca": range(1,N+1), "imię": imie, "nazwisko": nazwisko, 
                         "wiek": wiek, "adres": adres, "telefon": tel})

In [4]:
def transform_wiek(wiek): 
    """
    Funkcja zwracająca częstość z jaką dana osoba na podstawie wieku mogłaby nas odwiedzać. 
    Im większa wartość tym większe prawdopodobieństwo wizyty.
    
    """
    
    if wiek < 10:
        return 1
    elif wiek < 12:
        return 2
    elif wiek < 16:
        return 3
    elif wiek < 20:
        return 5
    elif wiek < 30:
        return 6
    elif wiek < 38:
        return 5
    elif wiek < 45:
        return 4
    elif wiek < 55:
        return 3
    elif wiek < 70:
        return 2
    else:
        return 1


In [5]:
def inven(n = 100):
    """
    Funkcja tworząca prosty magazyn (wynajem/sklep). Zwraca ile gier o danym tytule (id_tytułu) jest u nas dostępna na półce.
    n = 100 - bo tyle różnych gier (i takich id) mamy w asortymencie
    """
    
    ilosc = np.random.randint(20, 30, n) ## od 20 do 30 każdego z tytułów
    ids = range(1, (n+1))
    ilosci = [ids[x] for x in range(len(ids)) for i in range(ilosc[x])]
    
    return pd.DataFrame({"id_inv": range(1, (len(ilosci)+1)),"id_tytuł": ilosci })

In [6]:
ludzie(im, ik, nm, nk, ulice_jaworze)

,id_mieszkańca,imię,nazwisko,wiek,adres,telefon
0,1,Genowefa,Kozłowska,54,Nadbrzeżna 82,681915037
1,2,Mariusz,Kaczmarek,34,Brzozowa 26,554725611
2,3,Oliwia,Maciejewska,8,NULL,863848844
3,4,Wioletta,Sawicka,59,Mała 45,901067382
4,5,Mykola,Szczepaniak,36,Niewielka 71,991120654
...,...,...,...,...,...,...
7344,7345,Igor,Kania,54,Miętowa 80,767374308
7345,7346,Zdzisław,Sokołowski,16,Relaksowa 47,965924070
7346,7347,Mieczysław,Baranowski,91,Nad Polami 98,509294147
7347,7348,Michalina,Olszewska,15,Świerkowa 78,612947099


In [110]:
np.random.randint(2,6)

4

In [169]:
def odwiedziny(start = datetime.datetime(2022, 6, 6, 9, 0, 0), lamb = 4):
    """
    Funkcja symulująca odwiedziny w naszym sklepie - sprzedaże, wynajmy, a także zakup w outlecie. 
    Zwraca godziny wizyt, mieszkańców, co kupili/ wynajęli.
    start - dzień startowy naszego przedsiębiorstwa, niech to będzie 6 czerwca 2022, 9:00
    lamb - lambda - parametr potrzebny do generowania procesu Poissona - zakładamy 4 klientów na godzinę w naszym sklepie
   
    """
    
    mieszkancy = ludzie(im, ik, nm, nk, ulice_jaworze) #baza mieszkańców, którzy mogą nas odwiedzić
    T = 239/30 #7h 58 min bo tyle nasz sklep jest otwarty (z małym buforem, żeby nikt nie kupił czegoś po 1 s od otwarcia)
    
    now = datetime.datetime.now()
    days = (now - start).days ##żeby te daty były do dzisiaj
    
    
    inv_ren = inven() ##startowy magazyn wynajmu
    inv_shop = inven() ##startowy magazyn sklepu
    pp = [transform_wiek(mieszkancy["wiek"][x]) for x in range(len(mieszkancy))] ##wagi dla mieszkancow do losowania
    
    df = pd.DataFrame({}) ##podstawowa tabela, do której dodawane będą wizyty
    rental =  pd.DataFrame({"id_inv": [], "return_date": [], "zniszczona": []}) ##tabela pomocnicza dla inv rentalowego
    kupno =  pd.DataFrame({"id_inv": [], "kupno": [], "wizyta":[]}) ##dla kupna
    outlet = pd.DataFrame({"id_outlet":[], "id_inv": [], "id_gry": [], "return_date": []}) ##dla outletu 
    arch_outlet =  pd.DataFrame({"id_inv": [], "date": []})
    
    swieta = [datetime.datetime(2023, 1, 6, 9, 0, 0), datetime.datetime(2023, 4, 9, 9, 0, 0), 
              datetime.datetime(2023, 4, 10, 9, 0, 0), datetime.datetime(2023, 3, 1, 9, 0, 0), 
              datetime.datetime(2023, 3, 3, 9, 0, 0), datetime.datetime(2023, 6, 8, 9, 0, 0), 
              datetime.datetime(2022, 6, 16, 9, 0, 0), datetime.datetime(2022, 8, 15, 9, 0, 0),
              datetime.datetime(2022, 11, 1, 9, 0, 0), datetime.datetime(2022, 11, 11, 9, 0, 0),
              datetime.datetime(2022, 12, 25, 9, 0, 0), datetime.datetime(2022, 12, 26, 9, 0, 0),]
    dd = [(s - start).days + 1 for s in swieta] ##dni jakie minęły od startu do święta (żeby móc je ominąć łatwo)
    
    exp = np.ceil(np.random.exponential(1, 150)) ##z tego będą losowane ilości gier kupionych/wynajętych
    id_out = 1 ##początkowy indeks dla outletu
    
    for d in range(1, days + 1):
        
        if d%6 == 0 or d%7 == 0 or (d in dd): ## wyłączenie weekendów oraz świąt
            pass
        
        else:
            ##odwiedziny zgodne z procesem poissona
            lamb = np.random.randint(2,6)
            N = np.random.poisson(T*lamb) ##ilość osób na cały dzień
            ts = sorted(T*np.random.uniform(0,1, N)) ##czasy wizyt (takie surowe)
            t_wizyty = [(start + datetime.timedelta(minutes=1) + datetime.timedelta(
                minutes=round(t*60,1))) for t in ts] #.strftime("%m/%d/%Y %H:%M:%S") for t in ts]
            ##formatka wizyt
            
            for t in t_wizyty:
                ##losowanie klientów z populacji mieszkańców
                klient = random.choices(mieszkancy["id_mieszkańca"], weights=pp , k=1)
            
                #losowanie sprzedawcy (zakładamy, że sprzedawca 1 i 2 obsługują kasy przez te 8 h)
                sprzedawca = random.choices([1,2], k=1)
            
                ##ktoś może chce kupić kilka gier zamiast jednej
                ilosc = random.choices(exp, k = 1)
            
                ##ktoś przychodzi z zamiarem kupna/wynajmu bądź skorzystania z outletu (jeśli może)
                ##ilość dostępnych gier w outlecie musi wynosić min tyle ile ktoś chce tych gier zakupić
                
                if len(outlet[outlet["return_date"] < start]) >= ilosc[0]: 
                    rodzaj = random.choices(["wynajem", "kupno", "outlet"], weights = [10, 10, 2], k = 1)
                    ##ustalamy prawdopodobieństwa skorzystania z outletu na 2:10:10 względem zwykłego kupna i wynajmu
                else:
                    rodzaj = random.choices(["wynajem", "kupno"], k = 1)
                    #jeśli outlet jest za słabo wyposażony zostaje tylko opcja wynajmu / kupna z wagami 1:1
                
                if rodzaj == ["wynajem"]:
                    
                    ##losuje zwrot za 1 lub 2 dni w godzinach otwarcia, bo zakładamy, że tyle mają czasu na zwrot
                    return_date = start + datetime.timedelta(days=random.randint(1,2)) + datetime.timedelta(
                        hours=round(random.uniform(1,T), 2))
                    
                    ##losuje id_gry, które ktoś wypożycza
                    ##szukamy tych id które są dostępne, czyli NIE ma ich w df rental z returndate > dziś
                    
                    not_in_use = np.array(rental[rental["return_date"] > start]["id_inv"]) ##niedostępne
                    ids = np.array(inv_ren["id_inv"]) ##wszystkie
                    in_use = np.setdiff1d(ids, not_in_use) ##różnica między wszystkimi a tymi niedostępnymi
                    gra = random.sample(sorted(in_use), k = int(ilosc[0])) ##z tych dostepnych losowanie gry
                    
                    ##ewentualne zniszczenia zwróconej gry - prawdopodobieństwo ustalamy na 1/100
                    fault = random.choices([0,1], weights = [99,1], k = int(ilosc[0])) 
                    
                    ##wrzucenie informacji do tymczasowej tabeli rental
                    rent_gry =  pd.DataFrame({"id_inv": gra, "return_date": np.repeat(return_date, int(ilosc[0])), 
                                              "zniszczona": fault})
                    rental = pd.concat([rental, rent_gry])
                    
                    ## gra może zostać zniszczona, a więc może trafić do outletu
                    if sum(fault) > 0: ##czy były jakieś zniszczenia - jeśli tak to update outletu
                        
                        idd = rent_gry[rent_gry["zniszczona"] == 1]["id_inv"] ##te zniszczone id_inv
                        
                        for i in idd: ##dla każdej zniszczonej gry
                            ##znajdujemy id_tytułu (bo działaliśmy na inv)
                            id_tyt = inv_ren.loc[inv_ren['id_inv'] == i]["id_tytuł"] 
                
                            f = pd.DataFrame({"id_outlet": id_out, "id_inv": i, "id_gry": id_tyt, 
                                              "return_date": return_date})
                            outlet = pd.concat([outlet, f]) ## i dodajemy gre do outletu
                            id_out += 1
                            ###print("----", outlet, start)
                    
                elif rodzaj == ["kupno"]: 
                    ##to samo co z rentalem, tylko patrzymy na id gry, które ktoś kupił DZISIAJ 
                    ##bo zakładamy, że kupiona dzisiaj gra, dostępna będzie ponownie jutro
                    
                    return_date = None
                    not_in_use2 = np.array(kupno[kupno["kupno"] == start]["id_inv"])
                    ids2 = np.array(inv_shop["id_inv"])
                    in_use2 = np.setdiff1d(ids2, not_in_use2)
                    gra = random.sample(sorted(in_use2), k = int(ilosc[0]))
                
                    kup_gry =  pd.DataFrame({"id_inv": gra, "kupno": np.repeat(start, int(ilosc[0])),
                                             "wizyta": np.repeat(t, int(ilosc[0]))})
                    kupno = pd.concat([kupno, kup_gry])
                    
                else: ##outlet
                    return_date = None
                    in_use = outlet[outlet["return_date"] < start] ##patrzymy na te już zwrócone i dostepne gry
                    gra = random.sample(sorted(in_use["id_inv"]), k = int(ilosc[0])) ##losowanie z dostepnego outletu
                    ##id_inv odnoszą się na razie do magazynu wynajmu, ale później odnosić się będą do swojego id
                    ##to w formatce tabel
                    
                    for g in gra:
                        outlet = outlet.drop(outlet.index[outlet["id_inv"] == g])
                        ##usuwanie z outletu kupionych pozycji ^
                        arch = pd.DataFrame({"id_inv": g, "date": t}, index=[0])
                        arch_outlet = pd.concat([arch_outlet, arch]) ##wrzucamy do archiwum transakcji
                        
                ##dołączenie danego klienta do df z poprzednich dni
                day = pd.DataFrame({"id_mieszkańca": klient, "wizyta": t,
                                "sprzedawca": sprzedawca, "ilosc": int(ilosc[0]), "id_gry": str(gra)[1:-1], "rodzaj": rodzaj,
                                   "return_date": return_date})
                df = pd.concat([df, day])
                     
        ##przejście z datą na kolejny dzień
        start = start + datetime.timedelta(days=1)
        
    return df, rental, kupno, outlet, inv_ren, inv_shop, mieszkancy, arch_outlet

In [170]:
def format_tabel(wizyty, gry = gry):
    """
    Funkcja służąca do formatowania otrzymanych wyników z funkcji odwiedziny() w taki sposób, żeby zwracała
    gotowe tabele, które trafią do bazy danych.
    wizyty - wynik z funkcji odwiedziny()
    gry - tabela z grami naszego wykonania
    
    """
    tabela_g = wizyty[0] ##tabela główna
    rental_arch = wizyty[1] ##archiwum wynajmu
    shop_arch = wizyty[2] ##archiwum sprzedaży
    outlet_akt = wizyty[3] ##aktualny outlet
    rent_inv = wizyty[4] ##inventory rental
    shop_inv = wizyty[5] ##inventory shop
    mieszkancy = wizyty[6] ##cała społeczność
    outlet_arch = wizyty[7] ## archiwum outletu

    tabela_g = tabela_g.rename(columns = {"sprzedawca":"id_pracownika"})
    ## ------------- TABELA KLIENCI -------------
    
    #szukamy unikatowych osób, które pojawiły się u nas w sklepie i ich ostatnią wizytę
    klient = tabela_g.drop_duplicates(subset=['id_mieszkańca'], keep="last")[["id_mieszkańca", "wizyta"]]
    klienci = pd.merge(klient, mieszkancy, how = "inner", on = ["id_mieszkańca"]) ##dopasowanie względem id_mieszkanca
    klienci["id_klienta"] = range(1, len(klienci) + 1) ##stworzenie id klienta
    
    klienci_mieszkancy = klienci[["id_klienta", "id_mieszkańca"]] ##tabela pomocnicza z id klienta dopasowanymi do mieszkańca
    
    klienci = klienci[["id_klienta", "wizyta", "imię", "nazwisko", "wiek", "adres", "telefon"]] #końcowa tabela klientów
    
    ## ---------------- TABELA SPICHLERZ_WYNAJEM ---------------
    ceny = gry[["id_gry","cena"]]
    ceny = ceny.rename(columns = {"id_gry": "id_tytuł"}) ##tymczasowa tabela z cenami
    ceny["cena_wynajem"] = np.round(0.15*ceny["cena"], 2 ) ###bo ustalamy, że cena wynajmu gry kosztuje 15% jej ceny rynkowej
    
    one = pd.merge(rent_inv, ceny, on="id_tytuł") ##krok pierwszy - połączenie wynajmu z cenami najmu
    ids = np.array(rental_arch["id_inv"])
    rental_arch["id_inv"] = [int(i) for i in ids]  ##po random sample zostały float jako typ id, więc wracam na int
    two = rental_arch.drop_duplicates(subset=['id_inv'], keep='last') ##biorę pod uwagę tylko ostatnie wypożyczenie danej gry
    
    spichlerz_wynajemt = pd.merge(one,
                             two[["id_inv","return_date"]], 
                             on=["id_inv"], how="left").rename(columns = 
                                                               {"id_inv": "id_spichlerz_wynajem", "id_tytuł": "id_gry", "return_date": "ostatni_update"})
    ##i ostateczna tabela:
    spichlerz_wynajem = spichlerz_wynajemt[["id_spichlerz_wynajem", "id_gry", "ostatni_update"]]
    
    ## ------------------- TABELA SPICHLERZ_SKLEP ---------------
    
    ceny["cena_kupno"] = np.round(1.1*ceny["cena"], 2 ) ###bo ustalamy marżę 10% na zakupie
    raz = pd.merge(shop_inv, ceny, on="id_tytuł")[["id_tytuł", "cena_kupno", "id_inv"]] ##znowu pierwsze merge z cenami
    
    ids2 = np.array(shop_arch["id_inv"])
    shop_arch["id_inv"] = [int(i) for i in ids2] ##znowu zmiana na int
    shop_arch["kupno"] = [d + datetime.timedelta(days=1) for d in shop_arch["kupno"]] ## bo gra jest dostepna 1 dzień po zakupie
    dwa = shop_arch.drop_duplicates(subset=['id_inv'], keep='last') ##znowu patrzymy na ostatnie zakupy
    spichlerz_sklept = pd.merge(raz,
                             dwa[["id_inv","kupno"]], 
                             on=["id_inv"], how="left").rename(columns = 
                                                               {"id_inv": "id_spichlerz_sklep", "id_tytuł": "id_gry", "kupno": "ostatni_update"})
    spichlerz_sklep = spichlerz_sklept[["id_spichlerz_sklep", "id_gry", "ostatni_update"]] ##ostatnia formatka
    
    ## ---------------- TABELA WYNAJEM ---------
    sub = pd.merge(rental_arch, tabela_g, on = ["return_date"])[["id_inv", "wizyta", "return_date", "zniszczona", "id_mieszkańca", "id_pracownika"]]
    ##łączymy archiwum z główną tabelą - mamy teraz więcej informacji i dodajemy id klienta
    wynajemt = pd.merge(sub, klienci_mieszkancy, how = "left", on = ["id_mieszkańca"]).rename(columns={"id_inv":"id_spichlerz_wynajem"})
    ## i merge tabeli tymczasowej ze spichlerzem, żeby uzyskać cenę
    wynajem = pd.merge(wynajemt, spichlerz_wynajemt[["id_spichlerz_wynajem","cena_wynajem"]], on=["id_spichlerz_wynajem"], how="left")
    ##i ostateczna forma tabeli:
    wynajem =  wynajem.sort_values(by = "wizyta", inplace=False, ascending=True) ##sort po datach
    wynajem = wynajem.rename(columns = {"wizyta": "data_wynajmu", "return_date": "data_zwrotu"}) ##szybki rename
    wynajem["id_transakcji_wynajem"] = range(1, len(wynajem) + 1) ##dodanie id_transakcji
    wynajem = wynajem[["id_transakcji_wynajem", "id_spichlerz_wynajem","cena_wynajem", "data_wynajmu", "data_zwrotu", 
                       "id_pracownika", "id_klienta", "zniszczona"]]
    
    ## ----------- TABELA SKLEP --------------------
    
    shop_arch['wizyta'] = pd.to_datetime(shop_arch['wizyta']) ##merge z tabelą główną
    sklept = pd.merge(shop_arch, tabela_g, on="wizyta", how="left")[["id_inv", "wizyta", "id_mieszkańca", "id_pracownika"]]
    sklep = pd.merge(sklept, klienci_mieszkancy, how="left", on = ["id_mieszkańca"]).rename(columns = {"id_inv": "id_spichlerz_sklep"})
    sklep["id_transakcji_sklep"] = range(1, len(sklep) + 1)
    sklep = pd.merge(sklep, spichlerz_sklept[["id_spichlerz_sklep", "cena_kupno"]], on="id_spichlerz_sklep", how="left")
    sklep = sklep[["id_transakcji_sklep", "id_spichlerz_sklep", "wizyta", "cena_kupno", "id_klienta", "id_pracownika"]]
    
    
    ## -------------- TABELA OUTLET ----------------
    
    outlet_arch['date'] = pd.to_datetime(outlet_arch['date'])
    ids = outlet_arch["id_inv"]
    outlet_arch["id_inv"] = [int(i) for i in ids]
    outlet_arch = outlet_arch.rename(columns = {"id_inv":"id_spichlerz_wynajem", "date":"wizyta"})
    out = pd.merge(outlet_arch, tabela_g, on="wizyta", how="left")[["id_spichlerz_wynajem", "wizyta", "id_mieszkańca", "id_pracownika"]]
    out2 = pd.merge(out, spichlerz_wynajemt[["id_spichlerz_wynajem", "id_gry", "cena_wynajem"]], on="id_spichlerz_wynajem", how="left")
    out2["cena_outlet"] = [3* c for c in out2["cena_wynajem"]] 
    ##bo ustalamy 45% czyli 3* cenę najmu jako cenę kupna outletowej gry
    outlett = pd.merge(out2, klienci_mieszkancy, how="left", on = ["id_mieszkańca"])
    outlett["id_transakcji_outlet"] = range(1, len(outlett) + 1)
    outlet = outlett[["id_transakcji_outlet", "id_spichlerz_wynajem", "id_gry","cena_outlet", 
                      "wizyta", "id_klienta", "id_pracownika"]]
    
    ## -------------------TABELA SPICHLERZ_OUTLET --------------
    spichlerz_outlet = outlet_akt.rename(columns={ "id_inv":"id_spichlerz_wynajem", "return_date":"data_zwrotu"})
    ids = spichlerz_outlet["id_spichlerz_wynajem"]
    spichlerz_outlet["id_spichlerz_wynajem"] = [int(i) for i in ids]
    ids = spichlerz_outlet["id_outlet"]
    spichlerz_outlet["id_outlet"] = [int(i) for i in ids]
    ids = spichlerz_outlet["id_gry"]
    spichlerz_outlet["id_gry"] = [int(i) for i in ids]
    ##data zwrotu czyli dostępnosci
    
    return klienci, spichlerz_wynajem, spichlerz_sklep, wynajem, sklep, outlet, spichlerz_outlet

In [172]:
tt = odwiedziny()

----       id_outlet  id_inv  id_gry          return_date
2275        1.0  2276.0    94.0  2022-06-15 15:23:24 2022-06-14 09:00:00
----       id_outlet  id_inv  id_gry          return_date
2275        1.0  2276.0    94.0  2022-06-15 15:23:24
196         2.0   197.0     9.0  2022-06-20 11:03:36 2022-06-18 09:00:00
----       id_outlet  id_inv  id_gry          return_date
1325        3.0  1326.0    55.0  2022-07-08 13:47:24 2022-07-07 09:00:00
----       id_outlet  id_inv  id_gry          return_date
1325        3.0  1326.0    55.0  2022-07-08 13:47:24
623         4.0   624.0    26.0  2022-07-08 14:25:48 2022-07-07 09:00:00
----      id_outlet  id_inv  id_gry          return_date
130        5.0   131.0     6.0  2022-07-21 11:34:48 2022-07-20 09:00:00
----       id_outlet  id_inv  id_gry          return_date
130         5.0   131.0     6.0  2022-07-21 11:34:48
1718        6.0  1719.0    71.0  2022-07-26 14:23:24 2022-07-25 09:00:00
----       id_outlet  id_inv  id_gry          return_date

In [173]:
tab = format_tabel(tt)
klienci = tab[0]
spichlerz_wynajem = tab[1]
spichlerz_sklep = tab[2]
wynajem = tab[3]
sklep = tab[4]
outlet = tab[5]
spichlerz_outlet = tab[6]

In [174]:
outlet

,id_transakcji_outlet,id_spichlerz_wynajem,id_gry,cena_outlet,wizyta,id_klienta,id_pracownika
0,1,2276,94,9.00,2022-06-18 14:39:24,78,1
1,2,197,9,63.00,2022-06-22 09:45:36,3627,1
2,3,1326,55,63.00,2022-07-09 16:06:42,191,2
3,4,624,26,81.00,2022-07-09 16:41:36,1345,2
4,5,131,6,193.50,2022-07-26 11:09:06,281,2
5,6,1719,71,31.50,2022-07-27 09:55:06,290,2
6,7,1086,45,45.00,2022-07-27 11:59:18,294,1
7,8,2336,97,72.00,2022-08-02 09:03:24,321,1
8,9,1463,61,58.50,2022-08-02 09:03:24,321,1
9,10,2179,90,33.75,2022-08-17 12:32:48,401,1


In [175]:
len(wynajem[wynajem["zniszczona"] == 1])

49

In [176]:
spichlerz_outlet

,id_outlet,id_spichlerz_wynajem,id_gry,data_zwrotu
108,48,109,6,2023-06-22 10:16:48
1121,49,1122,47,2023-06-23 13:40:12


In [177]:
len(outlet)

47

##  tabele sprzedawców

In [53]:
def payment(rola):
    if rola == "sprzedawca":
        return random.randint(3700, 4300)
    elif rola == "logistyk":
        return random.randint(5000, 6000)
    else:
        return random.randint(6000, 7000)

In [54]:
def sales(ul = ulice_jaworze):
    role = ["sprzedawca", "sprzedawca", "manager", "logistyk"]
    id_pracownika = [1,2,3,4]
    pay_s = payment("sprzedawca")
    pay_m = payment("manager")
    pay_l = payment("logistyk")
    pensje = [pay_s, pay_s, pay_m, pay_l]
    
    imiona = ["Misio", "Nati", "Alutka", "Fifol"]
    nazwiska = ["Bresiński", "Lach", "Myśliwiec", "Oszczepaliński"]
    wiek = [22, 22, 21, 21]
    r = random.sample(id_pracownika, k=4)
    
    adresy = [random.choice(ul) + ' ' + str(random.randint(1,100)) for _ in range(4)]
    telefony = [random.randint(500000000, 999999999) for _ in range(4)]
    
    one = pd.DataFrame({"id_pracownika" : r, "imię_pracownika" : imiona, 
                        "nazwisko_pracownika" : nazwiska, "wiek_pracownika": wiek,
                       "adres_pracownika" : adresy, "telefon_pracownika" : telefony})
    two = pd.DataFrame({"id_pracownika": id_pracownika, "rola": role, "pensja":pensje})
    
    pracownicy = pd.merge(one, two, on="id_pracownika")
    pracownicy = pracownicy.sort_values(by="id_pracownika")
    
    return pracownicy

In [122]:
pracownicy = sales()
pracownicy

,id_pracownika,imię_pracownika,nazwisko_pracownika,wiek_pracownika,adres_pracownika,telefon_pracownika,rola,pensja
0,1,Misio,Bresiński,22,Cyprysowa 67,671149042,sprzedawca,4100
1,2,Nati,Lach,22,Irysów 45,933317240,sprzedawca,4100
3,3,Fifol,Oszczepaliński,21,Poprzeczna 84,577579855,manager,6879
2,4,Alutka,Myśliwiec,21,Niewielka 1,517200955,logistyk,5046


In [178]:
from sqlalchemy import create_engine, inspect
from sqlalchemy import URL

url_object = URL.create(
    "mysql+pymysql",
    username="team26",
    password="***",  # tu wpisać hasło i go nie commitować bo gitguardian was zje
    host="giniewicz.it",
    database="team26",
)

engine = create_engine(url_object)#'mysql+pymysql://team26:te@mzg@giniewicz.it:3306/team26')

connection = engine.connect()

connection.close()

In [179]:
conn = engine.connect()
gry.to_sql("gry", engine, if_exists="replace", index = False)
pracownicy.to_sql("pracownicy", engine, if_exists="replace", index = False)
klienci.to_sql("klienci", engine, if_exists="replace", index=False)
outlet.to_sql("outlet", engine, if_exists="replace", index=False)
spichlerz_wynajem.to_sql("spichlerz_wynajem", engine, if_exists="replace", index=False)
wynajem.to_sql("wynajem", engine, if_exists="replace", index=False)
sklep.to_sql("sklep", engine, if_exists="replace", index=False)
spichlerz_sklep.to_sql("spichlerz_sklep", engine, if_exists="replace", index=False)
spichlerz_outlet.to_sql("spichlerz_outlet", engine, if_exists="replace", index=False)
conn.close()

-----
# TURNIEJE
--- 

In [11]:
def turniej(gracze,sedzia,gry = gry):
    start = datetime.datetime(2022, 6, 9, 18, 0, 0)
    now = datetime.datetime.now()
    days = (now - start).days ##żeby te daty były do dzisiaj
    two_weeks = datetime.timedelta(weeks = 2)

    date_tournament = np.array([start])
    while date_tournament[-1]<now:
        next_tournament = date_tournament[-1]+two_weeks
        date_tournament = np.append(date_tournament,next_tournament)

    swieta = [datetime.datetime(2023, 1, 6, 18, 0, 0), datetime.datetime(2023, 4, 18, 9, 0, 0), 
              datetime.datetime(2023, 4, 10, 18, 0, 0), datetime.datetime(2023, 3, 1, 18, 0, 0), 
              datetime.datetime(2023, 3, 3, 18, 0, 0), datetime.datetime(2023, 6, 8, 18, 0, 0), 
              datetime.datetime(2022, 6, 16, 18, 0, 0), datetime.datetime(2022, 8, 15, 18, 0, 0),
              datetime.datetime(2022, 11, 1, 18, 0, 0), datetime.datetime(2022, 11, 11, 18, 0, 0),
              datetime.datetime(2022, 12, 25, 18, 0, 0), datetime.datetime(2022, 12, 26, 18, 0, 0),]

    date_tournament = np.delete(date_tournament,np.isin(date_tournament,swieta))
### turniej
    gra_tournament = gry[gry['TURNIEJOWE'] == 1]
    id_turniej = [i for i in range(1,len(gra_tournament)+1)]
    # zakładam, iz czsas dodatkowy to 0.3 czas podstawowego plus 5 minut przerwy między zmianami
    amount = np.floor(240/(np.array(gra_tournament[['CZAS_GRY']])+np.array(gra_tournament[['CZAS_GRY']])*0.3))*2
    amount = amount.transpose()
    mean_point = np.array([110,50,350])
    #mean_point = mean_point.transpose()
    turniej = pd.DataFrame({'id_turnieju':id_turniej,'id_gry':gra_tournament['ID'],'średnia_punktów' : mean_point, 'ilosc_gier' : amount[0]})
# rozgrywka
    id_rozgrywki = [i for i in range(1,len(date_tournament)+1)]
    kind_tournament = random.choices(np.array(id_turniej),k=len(date_tournament))
    date_date = [dt.date() for dt in date_tournament]
    rozgrywka = pd.DataFrame({'id_rozgrywki':id_rozgrywki, 'id_turniej':kind_tournament,'data':date_date})
# wyniki
    id_rozgrywekk = np.array([])
    id_klientas = np.array([])
    wyniki = np.array([])
    czas_rozrywki = np.array([])
    for _ in range(len(date_tournament)):

        kt = kind_tournament[_]
        ig = turniej[turniej['id_turnieju']==kt]['id_gry'].iloc[0]
        il = np.array(gry[gry['ID']==ig][['MIN_GRACZY','MAX_GRACZY']])
        il_max = il[0][1]
        #n = sum([random.randint(il[0][0],il[0][1]) for i in range(int(turniej[turniej['id_turnieju']==kt][['ilosc_gier']].iloc[0]))])
        #n = int(il[0][1]*turniej[turniej['id_turnieju']==kt]['ilosc_gier'].iloc[0])
        n = int(turniej[turniej['id_turnieju']==kt]['ilosc_gier'].iloc[0]/2)

        for n_ in range(n):

            id_rozgrywki_s = np.round(np.array([id_rozgrywki[_]]*(2*il_max)),0)
            id_rozgrywekk = np.append(id_rozgrywekk,id_rozgrywki_s)

            id_klienta = random.choices(np.array(gracze['id_klienta']),k = il_max*2)
            id_klientas = np.append(id_klientas,id_klienta)

            wynik = [turniej[turniej['id_turnieju']==kt]['średnia_punktów'].iloc[0]]*il_max*2 + np.random.normal(scale=turniej[turniej['id_turnieju']==kt]['średnia_punktów'].iloc[0]*0.15,size=il_max*2)
            wyniki = np.append(wyniki,wynik)

            czas_1 = np.round([gra_tournament[gra_tournament['ID']==ig]['CZAS_GRY'].iloc[0]*il_max + np.random.normal(scale=10)]*il_max,0)
            czas_2 = np.round([gra_tournament[gra_tournament['ID']==ig]['CZAS_GRY'].iloc[0]*il_max + np.random.normal(scale=10)]*il_max,0)

            czas = np.append(czas_1,czas_2)
            czas_rozrywki = np.append(czas_rozrywki,czas)

    print(len(id_rozgrywekk),len(id_klientas),len(wyniki),len(czas_rozrywki))
    df_wynik = pd.DataFrame({'id_rozgrywki':id_rozgrywekk.astype(int),'id_klienta':id_klientas.astype(int),'wynik':np.ceil(wyniki).astype(int),'czas_rozgrywki':czas_rozrywki})    
    return turniej, rozgrywka, df_wynik  

In [60]:
n, m,p = turniej(klienci,0)

1064 1064 1064 1064


In [61]:
p

,id_rozgrywki,id_klienta,wynik,czas_rozgrywki
0,1,1227,331,370.0
1,1,4055,369,370.0
2,1,1334,288,370.0
3,1,1865,419,370.0
4,1,4180,383,356.0
...,...,...,...,...
1059,28,46,322,360.0
1060,28,2910,311,364.0
1061,28,4129,303,364.0
1062,28,3662,406,364.0
